In [11]:
using DifferentialEquations
using Plots

# Высокое качество графиков
gr(size=(1000, 800), dpi=300, legendfontsize=12, tickfontsize=10, guidefontsize=12)

# Параметры модели для 13 варианта
const N = 667
const n0 = 6

# Модель 1: dn/dt = (0.77 + 0.00017*n)*(N - n)
function model1!(du, u, p, t)
    du[1] = (0.77 + 0.00017*u[1])*(N - u[1])
end

# Модель 2: dn/dt = (0.000017 + 0.57*n)*(N - n)
function model2!(du, u, p, t)
    du[1] = (0.000017 + 0.57*u[1])*(N - u[1])
end

# Модель 3: dn/dt = (0.7*sin(2*t) + 0.5*cos(4*t)*n)*(N - n)
function model3!(du, u, p, t)
    du[1] = (0.7*sin(2*t) + 0.5*cos(4*t)*u[1])*(N - u[1])
end

# Решение моделей
println("Решаем модель 1...")
prob1 = ODEProblem(model1!, [n0], (0.0, 10.0))
sol1 = solve(prob1, Tsit5(), reltol=1e-8, abstol=1e-8)

println("Решаем модель 2...")
# Используем устойчивый метод для жесткой системы
prob2 = ODEProblem(model2!, [n0], (0.0, 0.8))
sol2 = solve(prob2, Rosenbrock23(), reltol=1e-10, abstol=1e-10, maxiters=1000000)

println("Решаем модель 3...")
prob3 = ODEProblem(model3!, [n0], (0.0, 10.0))
sol3 = solve(prob3, Tsit5(), reltol=1e-8, abstol=1e-8)

# Создаем графики
p1 = plot(sol1, label="n(t)", xlabel="Время t", ylabel="Количество n(t)", 
          title="Модель 1: dn/dt = (0.77 + 0.00017n)(667 - n)",
          linewidth=3, grid=true, gridstyle=:dash, legend=:bottomright)
hline!([N], label="N = 667", linestyle=:dash, linewidth=2, color=:red)

p2 = plot(sol2, label="n(t)", xlabel="Время t", ylabel="Количество n(t)", 
          title="Модель 2: dn/dt = (0.000017 + 0.57n)(667 - n)",
          linewidth=3, grid=true, gridstyle=:dash, legend=:bottomright)
hline!([N], label="N = 667", linestyle=:dash, linewidth=2, color=:red)

p3 = plot(sol3, label="n(t)", xlabel="Время t", ylabel="Количество n(t)", 
          title="Модель 3: dn/dt = (0.7sin(2t) + 0.5cos(4t)n)(667 - n)",
          linewidth=3, grid=true, gridstyle=:dash, legend=:bottomright)
hline!([N], label="N = 667", linestyle=:dash, linewidth=2, color=:red)

# Сохраняем отдельные графики
savefig(p1, "model1_variant13.png")
savefig(p2, "model2_variant13.png") 
savefig(p3, "model3_variant13.png")

# Общий график всех моделей
all_plot = plot(p1, p2, p3, layout=(3,1), size=(1000, 1200))
savefig(all_plot, "all_models_variant13.png")

# Анализ модели 2 - находим момент максимальной скорости
α1 = 0.000017
α2 = 0.57

# Аналитическое решение для максимума скорости
n_max_analytical = (α2*N - α1)/(2α2)
println("="^60)
println("АНАЛИЗ МОДЕЛИ 2 - ВАРИАНТ 13")
println("="^60)
println("Аналитический расчет:")
println("Максимум скорости достигается при n = ", round(n_max_analytical, digits=3))
println("Это соответствует половине рынка: N/2 = ", N/2)

# Численное определение момента максимальной скорости
speed_values = [(α1 + α2*u[1])*(N - u[1]) for u in sol2.u]
max_speed, max_idx = findmax(speed_values)
t_max = sol2.t[max_idx]
n_at_max = sol2.u[max_idx][1]

println("\nЧисленный результат:")
println("Максимальная скорость: ", round(max_speed, digits=2))
println("Момент времени: t = ", round(t_max, digits=6))
println("Количество информированных в этот момент: n = ", round(n_at_max, digits=2))

# График скорости для модели 2
p_speed = plot(sol2.t, speed_values, label="dn/dt - скорость", 
               xlabel="Время t", ylabel="Скорость dn/dt", 
               title="Скорость распространения рекламы: Модель 2",
               linewidth=3, grid=true, gridstyle=:dash,
               legend=:topright)
vline!([t_max], label="Максимум при t = $(round(t_max, digits=6))", 
       linestyle=:dash, linewidth=2, color=:red)
scatter!([t_max], [max_speed], label="Максимальная скорость", 
         color=:red, markersize=8)

savefig(p_speed, "speed_model2_variant13.png")

# Выводим все результаты
println("\n" + "="^60)
println("РЕЗУЛЬТАТЫ ДЛЯ ВАРИАНТА 13:")
println("="^60)
println("Модель 1: Доминирует платная реклама (α1 = 0.77)")
println("Модель 2: Доминирует 'сарафанное радио' (α2 = 0.57)") 
println("Модель 3: Переменные коэффициенты с колебаниями")
println("\nДля модели 2 максимальная скорость достигается при:")
println("Время: t ≈ ", round(t_max, digits=4))
println("Количество информированных: n ≈ ", round(n_at_max, digits=1))
println("Скорость распространения: ≈ ", round(max_speed, digits=1), " клиентов/ед.времени")

# Дополнительный анализ
println("\n" + "="^60)
println("ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ:")
println("="^60)
println("1. МОДЕЛЬ 1: Быстрый начальный рост благодаря сильной платной рекламе")
println("   (α1 = 0.77 >> α2 = 0.00017)")
println("2. МОДЕЛЬ 2: Классическая логистическая кривая")
println("   Максимум скорости при n ≈ N/2 = 333.5 (теория совпадает с расчетом)")
println("   Медленный старт, затем ускорение за счет 'сарафанного радио'")
println("3. МОДЕЛЬ 3: Сложное колебательное поведение")
println("   Из-за периодических коэффициентов sin(2t) и cos(4t)")

# Показываем все графики
display(p1)
display(p2) 
display(p3)
display(p_speed)
display(all_plot)

println("\nГрафики сохранены в файлы:")
println("- model1_variant13.png")
println("- model2_variant13.png")
println("- model3_variant13.png") 
println("- speed_model2_variant13.png")
println("- all_models_variant13.png")

Решаем модель 1...
Решаем модель 2...
Решаем модель 3...


┌ Warning: At t=0.611493891401273, dt was forced below floating point epsilon 1.1102230246251565e-16, and step error estimate = 1.1896872889955643. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/ISrX6/src/integrator_interface.jl:657


АНАЛИЗ МОДЕЛИ 2 - ВАРИАНТ 13
Аналитический расчет:
Максимум скорости достигается при n = 333.5
Это соответствует половине рынка: N/2 = 333.5

Численный результат:
Максимальная скорость: 63396.68
Момент времени: t = 0.012369
Количество информированных в этот момент: n = 333.58


LoadError: MethodError: no method matching +(::String, ::String)
The function `+` exists, but no method is defined for this combination of argument types.
String concatenation is performed with [36m*[39m (See also: https://docs.julialang.org/en/v1/manual/strings/#man-concatenation).

[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:596[24m[39m
[0m  +([91m::ChainRulesCore.ZeroTangent[39m, ::Any)
[0m[90m   @[39m [33mChainRulesCore[39m [90m~/.julia/packages/ChainRulesCore/Vsbj9/src/[39m[90m[4mtangent_arithmetic.jl:99[24m[39m
[0m  +(::Any, [91m::ChainRulesCore.NotImplemented[39m)
[0m[90m   @[39m [33mChainRulesCore[39m [90m~/.julia/packages/ChainRulesCore/Vsbj9/src/[39m[90m[4mtangent_arithmetic.jl:25[24m[39m
[0m  ...
